In [ ]:
example = """
###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
""".strip().splitlines()

In [ ]:
import numpy as np

mapping = {
    "#": 1,
    ".": 0,
    "S": 0,
    "E": 0,
}

def read(input):
    table = []
    s, e = None, None
    for i, row in enumerate(input):
        sj = row.find("S")
        ej = row.find("E")
        if sj != -1:
            s = i, sj
        if ej != -1:
            e = i, ej
        table.append([mapping[e] for e in row])
    return np.array(table), s, e

In [ ]:
dirs = [(1, 0), (0, 1), (-1, 0), (0, -1)]

def next_step(y, x, table, visited):
    for diry, dirx in dirs:
        ny = y + diry
        nx = x + dirx
        if 0 <= ny < table.shape[0] and 0 <= nx < table.shape[1] and not visited[ny, nx] and not table[ny, nx]:
            yield ny, nx


def bfs(table, s, e):
    visited = np.full_like(table, 0)
    steps = np.full_like(table, 0)
    v = [(*s, 0)]
    visited[*s] = 1
    finished = False
    while v and not finished:
        y, x, step = v[0]
        v = v[1:]
        for ny, nx in next_step(y, x, table, visited):
            visited[ny, nx] = 1
            v.append((ny, nx, step+1))
            steps[ny, nx] = step+1
            finished = (ny, nx) == e
    return steps

In [ ]:
def check_cheats(table, s, e, cheat_length=2, debug=False):
    steps = bfs(table, s, e)
    allcheats = np.array([], dtype=np.int64)
    for cl in range(2, cheat_length+1):
        cl_y = min(cl, steps.shape[0]-1)
        cheat1 = np.abs(steps[:steps.shape[0]-cl_y, :] - steps[cl_y:, :]) - cl_y
        table1 = np.logical_or(table[:table.shape[0]-cl_y, :], table[cl_y:, :])
        cheat1[table1] = 0
        cl_x = min(cl, steps.shape[1]-1)
        cheat2 = np.abs(steps[:, :steps.shape[1]-cl_x] - steps[:, cl_x:]) - cl_x
        table2 = np.logical_or(table[:, :table.shape[1]-cl_x], table[:, cl_x:])
        cheat2[table2] = 0
        cheats = np.concat((cheat1[cheat1 != 0], cheat2[cheat2 != 0]))
        allcheats = np.concat((allcheats, cheats))
    if debug:
        print("Steps\n", steps)
        print(allcheats)
        print(list(map(lambda x: (x[0].item(),x[1].item()),zip(*np.unique(allcheats, return_counts=True)))))
    return np.count_nonzero(allcheats >= 100)

In [ ]:
check_cheats(*read(example), debug=True)

In [ ]:
from pathlib import Path

input = Path("1.txt").read_text().splitlines()

In [ ]:
check_cheats(*read(input))

In [ ]:
check_cheats(*read(example), cheat_length=20, debug=True)

In [ ]:
check_cheats(*read(input), cheat_length=20)

In [ ]:
from collections import defaultdict

dirs2 = [(1, 0), (0, 1)]

def next_step2(sy, sx, y, x, steps, length, i=1):
    for diry, dirx in dirs2:
        ny = y + diry
        nx = x + dirx
        if 0 <= ny < steps.shape[0] and 0 <= nx < steps.shape[1]:
            if steps[ny, nx]:
                yield abs(steps[sy, sx]-steps[ny, nx])-i
            if i < length:
                yield from next_step2(sy, sx, ny, nx, steps, length, i+1)

def check_cheats2(table, s, e, cheat_length=20):
    steps = bfs(table, s, e)
    print(steps)
    freq = defaultdict(int)
    for y in range(table.shape[0]):
        for x in range(table.shape[1]):
            if steps[y, x]:
                for step in next_step2(y, x, y, x, steps, cheat_length):
                    freq[step.item()] += 1
    return sorted(freq.items(), key=lambda x: x[0])

In [ ]:
check_cheats2(*read(example), cheat_length=2)

In [ ]:
def check_cheats3(table, s, e, cheat_length=20):
    steps = bfs(table, s, e)
    visited = defaultdict(list)
    freq = defaultdict(int)
    for y1 in range(table.shape[0]):
        for x1 in range(table.shape[1]):
            for y2 in range(table.shape[0]):
                for x2 in range(table.shape[1]):
                    visited[(y1, x1)].append((y2, x2))
                    if (y1, x1) not in visited[(y2, x2)] and steps[y1, x1] and steps[y2, x2] and abs(y2-y1)+abs(x2-x1) <= cheat_length:
                        freq[abs(steps[y1, x1]-steps[y2, x2]).item()-abs(y2-y1)-abs(x2-x1)] += 1
    return sorted(freq.items(), key=lambda x:x[0])

In [ ]:
check_cheats3(*read(example), cheat_length=20)